#### 1. First of all, let's check given data.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files=os.path.join(dirname, filename)
        print(files)
        if files == "/kaggle/input/petfinder-pawpularity-score/train/7fc71b8da143721939715b1cfe22122f.jpg":
            break

In [ ]:
import pandas as pd
import numpy as np
train_data=pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
test_data=pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
import seaborn as sns
sns.distplot(train_data["Pawpularity"])

#### Look at the gauge 100. To cut outliers of datasets is essential to improve accuracy.

In [ ]:
#value=train_data["Pawpularity"].quantile(0.5)
#value2=train_data["Pawpularity"].quantile(0.17)
#print(value,value2)

In [ ]:
#train_data=train_data[train_data["Pawpularity"] < value]
#train_data=train_data[train_data["Pawpularity"] > value2]
#sns.distplot(train_data["Pawpularity"])

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt
stats.probplot(train_data["Pawpularity"], dist="norm", plot=plt)
plt.show()

In [ ]:
x_train=np.array(train_data.iloc[:,1:13])
x_test=np.array(test_data.iloc[:,1:])
y_train=np.array(train_data.iloc[:,13])

In [ ]:
#StandardScaler is essential for high accuracy.
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
x_train = stdsc.fit_transform(x_train)
x_test = stdsc.transform(x_test)

In [ ]:
from sklearn.model_selection import train_test_split
x_train2, x_valid2, y_train2, y_valid2 = train_test_split(x_train, y_train, test_size=0.1,random_state=123)

#### 1. XGBoost.

In [ ]:
import xgboost as xgb
xgbm2 = xgb.XGBRegressor(base_score=0.9, booster='gbtree', colsample_bylevel=1,
                         colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
                         importance_type='gain', interaction_constraints='',
                         learning_rate=0.100000012, max_delta_step=100, max_depth=800,
                         min_child_weight=1, monotone_constraints='()',
                         n_estimators=500, n_jobs=2, num_parallel_tree=1, random_state=1,
                         reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
                         tree_method='exact', validate_parameters=1, verbosity=None)
model1=xgbm2
model1.fit(x_train, y_train)
a1=model1.predict(x_valid2)
p1=model1.predict(x_test)
from sklearn.metrics import mean_squared_error
import numpy as np
w1=np.sqrt(mean_squared_error(a1,y_valid2))
w1

In [ ]:
sns.scatterplot(x=a1,y=y_valid2)

#### 2. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=123, max_depth=100,
                               min_samples_leaf=11, n_estimators=200,min_samples_split=2)
model2=forest
model2.fit(x_train, y_train)
a2=model2.predict(x_valid2)
p2=model2.predict(x_test)
w2=np.sqrt(mean_squared_error(a2,y_valid2))
w2

In [ ]:
sns.scatterplot(x=a2,y=y_valid2)

In [ ]:
#forest.get_params()

####  3.LightGBM

In [ ]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor()
gbm = lgb.LGBMRegressor(learning_rate=0.0001, max_depth=100, min_child_samples=20,
                        min_child_weight=0.001, num_leaves=31, reg_alpha=0, reg_lambda=1,random_state=0)
model3=gbm
model3.fit(x_train, y_train,verbose=True)
p3 = model3.predict(x_test)
a3= model3.predict(x_valid2)
w3=np.sqrt(mean_squared_error(a3,y_valid2))
w3

In [ ]:
sns.scatterplot(x=a3,y=y_valid2)

4.Deep Learning

In [ ]:
from sklearn.neural_network import MLPRegressor
deep_learning = MLPRegressor(hidden_layer_sizes=(100,100,100,100),random_state=123,verbose=True,activation='relu',
                             early_stopping=True,max_iter=500,
                             solver='adam',warm_start=True)
model4=deep_learning
model4.fit(x_train, y_train)
a4=model4.predict(x_valid2)
p4=model4.predict(x_test)
w4=np.sqrt(mean_squared_error(a4,y_valid2))
w4

In [ ]:
sns.scatterplot(x=a4,y=y_valid2)

##### 5.Support Vector Regressor

In [ ]:
from sklearn import svm
from matplotlib import pyplot as plt
svr = svm.SVR(C=30,epsilon=0.01,verbose=True,degree=100)
model7=svr
model7.fit(x_train,y_train)
p7 = model7.predict(x_test)
a7= model7.predict(x_valid2)
w7=np.sqrt(mean_squared_error(a7,y_valid2))
w7

In [ ]:
sns.scatterplot(x=a7,y=y_valid2)

In [ ]:
kinds=pd.concat([pd.DataFrame([w1],columns=["XGBoost"],index=["RMSE"]),pd.DataFrame([w2],columns=["R.Forest"],index=["RMSE"]),
                pd.DataFrame([w3],columns=["LightGBM"],index=["RMSE"]),pd.DataFrame([w4],columns=["Neural Net"],index=["RMSE"]),
                pd.DataFrame([w7],columns=["SVR Regressor"],index=["RMSE"])], axis=1).T.sort_values("RMSE")
kinds

In [ ]:
#submit_ans=pd.concat([test_data.iloc[:,[0]],epsilon_maker(1),pd.DataFrame(p1,columns=["XGBoost"]),pd.DataFrame(p2,columns=["R.Forest"]),pd.DataFrame(p3,columns=["LightGBM"])], axis=1)
submit_ans=pd.concat([test_data.iloc[:,[0]],pd.DataFrame(p1,columns=["XGBoost"]),pd.DataFrame(p2,columns=["R.Forest"]),pd.DataFrame(p3,columns=["LightGBM"]),pd.DataFrame(p4,columns=["Neural Net"]),pd.DataFrame(p7,columns=["SVR"])], axis=1)
submit_ans

In [ ]:
final=submit_ans.iloc[:,3:4].mean(axis=1)+1.8


In [ ]:
submit_ans_pd=pd.concat([test_data.iloc[:,[0]],pd.DataFrame(np.round(final,9),columns=["Pawpularity"])], axis=1)
submit_ans_pd

In [ ]:
submit_ans_pd.to_csv("submission.csv",index=False)
print("File Saved")

Thanks for checking my notebook :-)

<HR>